### Part I 
#### Data Cleaning, Wrandling, Additional information 

In [1]:
import pandas as pd
import os
import requests
import time

In [2]:
data = pd.read_csv("monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']

#Modifying data format
data['Month'] = data['Month'].apply(lambda x: str(x).zfill(2))
data['Day'] = data['Day'].apply(lambda x: str(x).zfill(2))

data['Hour'] = data['Hour'].fillna(0)
data['Hour'] = data['Hour'].astype(int).astype(str).str.zfill(4)


data['Date'] = data ['Year'].astype(str)+'-'+ data['Month'].astype(str)+ '-' + data['Day'].astype(str)
data = data[data['Date'] != '0000-00-00']

print(data.shape)
data.head()

(49005, 14)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll,Date
0,902363382,2015,01,05,Weekday,0000,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,01,06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,01,06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,01,07,Weekend,0900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,01,07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


In [3]:
data = data.rename(columns={'ll': 'Location'})
data.head()

,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Date
0,902363382,2015,01,05,Weekday,0000,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,01,06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,01,06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,01,07,Weekend,0900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,01,07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


In [4]:
# Subset only 2015 year

subset_df = data[data['Year'] == 2015]

print(subset_df.shape)
#subset_df.head(20)

#save subset as raw data
subset_df.to_csv('subset2015.csv', index=False)

(3835, 14)


In [5]:
#import subset2015 
df = pd.read_csv("subset2015.csv", encoding="unicode_escape")

print(df.shape)
df.head()

(3835, 14)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Date
0,902363382,2015,1,5,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,1,6,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,1,6,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,1,7,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,1,7,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


In [6]:
# Remove Year, Month and Day columns
df = df.drop(columns=['Year', 'Month', 'Day'])

#data.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

#df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)


df = df[['Master Record Number','Date', 'Weekend?', 'Hour', 'Collision Type', 'Injury Type', 'Primary Factor', 'Reported_Location', 'Latitude', 'Longitude', 'Location']]

df.head()

,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location
0,902363382,2015-01-05,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015-01-06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015-01-06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015-01-07,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015-01-07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


In [7]:
#save subset as raw data

df.to_csv('data2015.csv', index=False)

In [8]:
#import data2015 

df = pd.read_csv("data2015.csv", encoding="unicode_escape")

print(df.shape)
df.head()

(3835, 11)


,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location
0,902363382,2015-01-05,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015-01-06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015-01-06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015-01-07,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015-01-07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


In [9]:
data_500 = df.sample(n=500, random_state=40)

data_500.shape
data_500 = data_500.reset_index(drop=True)

# set the new RangeIndex as the new index for the DataFrame
data_500 = data_500.set_index(pd.RangeIndex(len(data_500)))

In [10]:
data_500.to_csv('data_500.csv', index=False)

In [11]:
df_500 = pd.read_csv('data_500.csv', encoding="unicode_escape")

df_500.head()

,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location
0,902582972,2015-12-04,Weekday,2200,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,2ND & HIGH,39.160944,-86.508288,"39.160944,-86.508288"
1,902484079,2015-07-06,Weekday,1300,2-Car,Incapacitating,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,S LEONARD SPRINGS,39.142048,-86.582592,"39.142048,-86.582592"
2,902399070,2015-02-06,Weekday,1600,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,BARGE & S LEONARD SPRINGS,39.135797,-86.582496,"39.13579732,-86.582496"
3,902403874,2015-02-06,Weekday,1300,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,GRAHAM & ROGERS,39.139920,-86.538416,"39.13992,-86.538416"
4,902471166,2015-06-06,Weekday,500,1-Car,No injury/unknown,ANIMAL/OBJECT IN ROADWAY,SR37N & WYLIE,39.198571,-86.529524,"39.19857096,-86.5295245"


In [12]:
df_500.shape

(500, 11)

In [13]:
#Additional Data from API 

def number_of_bars(row):
    
    api_key = 'OHBJD-6roYB0-z-lx_TwDukqiGPjGC2XtiE1SgeziEmMQDhWs26MfgUMRmsWB5DgHsR6M0NWfiLwlW9zihiXnDgCjsGW-Le67ybshU3_4YjIUThAMlm7mJ_4jOFBZHYx'
    headers = {'Authorization': f'Bearer {api_key}'}
    url_params = {
        'term': 'bars',        
        'limit': 50,
        'radius': 1500
    }

    url_params['latitude'] = row['Latitude']
    url_params['longitude'] = row['Longitude']

    res = requests.get(url='https://api.yelp.com/v3/businesses/search', headers=headers, params=url_params)
    data_bars = res.json()

    time.sleep(1)
    return data_bars['total']



In [14]:
df_500['Number of Bars'] = df_500.apply(number_of_bars, axis=1)

df_500.to_csv('data2015bars.csv', index=False)

df_500

KeyboardInterrupt: 

In [ ]:
def add_weather_condition(df, api_key):
    
    df['Weather Condition'] = None
    weather_condition = []

    for index, row in df.iterrows():
        q = f"{row['Latitude']},{row['Longitude']}"
        date = f"{row['Date']}"
        res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
        data_js = res.json()
        weather_data = data_js['data']['weather']
        
        for weather in weather_data:
            condition = weather['hourly'][0]['weatherDesc'][0]['value']
            weather_condition.append(condition)

    df.loc[:, 'Weather Condition'] = weather_condition
    
    print("done")

    time.sleep(1)

    return df

In [ ]:
api_key = 'de0c889d02a34f68a4603907232104'
df_500 = add_weather_condition(df_500, api_key)

df_500.to_csv('data2015bars_weather.csv', index=False)

df_500

done


,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Number of Bars,Weather Condition
0,902582972,2015-12-04,Weekday,2200,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,2ND & HIGH,39.160944,-86.508288,"39.160944,-86.508288",7,Clear
1,902484079,2015-07-06,Weekday,1300,2-Car,Incapacitating,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,S LEONARD SPRINGS,39.142048,-86.582592,"39.142048,-86.582592",1,Clear
2,902399070,2015-02-06,Weekday,1600,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,BARGE & S LEONARD SPRINGS,39.135797,-86.582496,"39.13579732,-86.582496",1,Clear
3,902403874,2015-02-06,Weekday,1300,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,GRAHAM & ROGERS,39.139920,-86.538416,"39.13992,-86.538416",2,Clear
4,902471166,2015-06-06,Weekday,500,1-Car,No injury/unknown,ANIMAL/OBJECT IN ROADWAY,SR37N & WYLIE,39.198571,-86.529524,"39.19857096,-86.5295245",0,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,902384628,2015-01-07,Weekend,1900,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,EAST RHORER & SOUTH WALNUT STREET,39.121440,-86.526496,"39.12144,-86.526496",0,Overcast
496,902405042,2015-03-01,Weekday,2200,3+ Cars,No injury/unknown,SPEED TOO FAST FOR WEATHER CONDITIONS,1ST & ROGERS,39.159504,-86.538592,"39.159504,-86.538592",41,Patchy moderate snow
497,902438510,2015-05-07,Weekend,1500,2-Car,No injury/unknown,IMPROPER TURNING,STATE ROAD 46,39.207648,-86.573792,"39.207648,-86.573792",0,Clear
498,902427112,2015-04-06,Weekday,1700,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,W THIRD,39.164752,-86.573104,"39.164752,-86.573104",5,Clear


In [ ]:
data_df = pd.read_csv("data2015bars_weather.csv", encoding="unicode_escape")

data_df

,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Number of Bars,Weather Condition
0,902582972,2015-12-04,Weekday,2200,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,2ND & HIGH,39.160944,-86.508288,"39.160944,-86.508288",7,Clear
1,902484079,2015-07-06,Weekday,1300,2-Car,Incapacitating,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,S LEONARD SPRINGS,39.142048,-86.582592,"39.142048,-86.582592",1,Clear
2,902399070,2015-02-06,Weekday,1600,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,BARGE & S LEONARD SPRINGS,39.135797,-86.582496,"39.13579732,-86.582496",1,Clear
3,902403874,2015-02-06,Weekday,1300,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,GRAHAM & ROGERS,39.139920,-86.538416,"39.13992,-86.538416",2,Clear
4,902471166,2015-06-06,Weekday,500,1-Car,No injury/unknown,ANIMAL/OBJECT IN ROADWAY,SR37N & WYLIE,39.198571,-86.529524,"39.19857096,-86.5295245",0,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,902384628,2015-01-07,Weekend,1900,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,EAST RHORER & SOUTH WALNUT STREET,39.121440,-86.526496,"39.12144,-86.526496",0,Overcast
496,902405042,2015-03-01,Weekday,2200,3+ Cars,No injury/unknown,SPEED TOO FAST FOR WEATHER CONDITIONS,1ST & ROGERS,39.159504,-86.538592,"39.159504,-86.538592",41,Patchy moderate snow
497,902438510,2015-05-07,Weekend,1500,2-Car,No injury/unknown,IMPROPER TURNING,STATE ROAD 46,39.207648,-86.573792,"39.207648,-86.573792",0,Clear
498,902427112,2015-04-06,Weekday,1700,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,W THIRD,39.164752,-86.573104,"39.164752,-86.573104",5,Clear


In [ ]:
def add_visability (df, api_key):
    df['Visibility (mi)'] = None
    visibility = []

    for index, row in df.iterrows():
        q = f"{row['Latitude']},{row['Longitude']}"
        date = f"{row['Date']}"
        res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
        data_js = res.json()
        weather_data = data_js['data']['weather']
    
        for weather in weather_data:
            vis = weather['hourly'][0]['visibilityMiles']
            visibility.append(vis)

    df.loc[:, 'Visibility (mi)'] = visibility

    print("Done.")

    time.sleep(1)

    return df

In [ ]:
api_key = 'de0c889d02a34f68a4603907232104'
gdf = add_visability (gdf, api_key)

gdf.to_csv('data2015visability.csv', index=False)

gdf

In [ ]:
def add_moon (df, api_key):
    df['Moon phase'] = None
    moon = []

    for index, row in df.iterrows():
        q = f"{row['Latitude']},{row['Longitude']}"
        date = f"{row['Date']}"
        res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
        data_js = res.json()
        weather_data = data_js['data']['weather']
    
        for weather in weather_data:
            moon = weather[0]['astronomy'][0]['moon_phase']
            moon.append(moon)

    df.loc[:, 'Moon phase'] = moon

    print("Done.")

    time.sleep(1)

    return df

In [ ]:
import pandas as pd
import os
import requests
import time

In [ ]:
def add_moon1(df, api_key):
    df['Moon phase'] = None

    for index, row in df.iterrows():
        q = f"{row['Latitude']},{row['Longitude']}"
        date = f"{row['Date']}"
        res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
        data_js = res.json()
        weather_data = data_js['data']['weather']

        moon_phases = []
        for weather in weather_data:
            if 'astronomy' in weather:
                moon_phase = weather['astronomy'][0]['moon_phase']
                moon_phases.append(moon_phase)

        if len(moon_phases) > 0:
            df.at[index, 'Moon phase'] = moon_phases[0]

    print("Done.")
    time.sleep(1)

    return df


In [ ]:
api_key = 'de0c889d02a34f68a4603907232104'

data_df = add_moon1(data_df, api_key)

data_df.to_csv('data2015moon.csv', index=False)


data_df

Done.


,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Number of Bars,Weather Condition,Moon phase
0,902582972,2015-12-04,Weekday,2200,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,2ND & HIGH,39.160944,-86.508288,"39.160944,-86.508288",7,Clear,Last Quarter
1,902484079,2015-07-06,Weekday,1300,2-Car,Incapacitating,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,S LEONARD SPRINGS,39.142048,-86.582592,"39.142048,-86.582592",1,Clear,Waning Gibbous
2,902399070,2015-02-06,Weekday,1600,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,BARGE & S LEONARD SPRINGS,39.135797,-86.582496,"39.13579732,-86.582496",1,Clear,Waxing Gibbous
3,902403874,2015-02-06,Weekday,1300,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,GRAHAM & ROGERS,39.139920,-86.538416,"39.13992,-86.538416",2,Clear,Waxing Gibbous
4,902471166,2015-06-06,Weekday,500,1-Car,No injury/unknown,ANIMAL/OBJECT IN ROADWAY,SR37N & WYLIE,39.198571,-86.529524,"39.19857096,-86.5295245",0,Clear,Waning Gibbous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,902384628,2015-01-07,Weekend,1900,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,EAST RHORER & SOUTH WALNUT STREET,39.121440,-86.526496,"39.12144,-86.526496",0,Overcast,Waxing Gibbous
496,902405042,2015-03-01,Weekday,2200,3+ Cars,No injury/unknown,SPEED TOO FAST FOR WEATHER CONDITIONS,1ST & ROGERS,39.159504,-86.538592,"39.159504,-86.538592",41,Patchy moderate snow,Waxing Gibbous
497,902438510,2015-05-07,Weekend,1500,2-Car,No injury/unknown,IMPROPER TURNING,STATE ROAD 46,39.207648,-86.573792,"39.207648,-86.573792",0,Clear,Waning Gibbous
498,902427112,2015-04-06,Weekday,1700,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,W THIRD,39.164752,-86.573104,"39.164752,-86.573104",5,Clear,Waxing Gibbous


In [ ]:
import pandas as pd
import os
import requests
import time

In [ ]:
api_key = 'de0c889d02a34f68a4603907232104'
q = "39.15920668,-86.52587356"
date = "2015-01-05"
res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
data_js = res.json()
data_js

{'data': {'request': [{'type': 'LatLon', 'query': 'Lat 39.16 and Lon -86.53'}],
  'weather': [{'date': '2015-01-05',
    'astronomy': [{'sunrise': '08:06 AM',
      'sunset': '05:38 PM',
      'moonrise': '06:30 PM',
      'moonset': '08:03 AM',
      'moon_phase': 'Full Moon',
      'moon_illumination': '100'}],
    'maxtempC': '-8',
    'maxtempF': '18',
    'mintempC': '-12',
    'mintempF': '11',
    'avgtempC': '-9',
    'avgtempF': '15',
    'totalSnow_cm': '0.0',
    'sunHour': '8.7',
    'uvIndex': '2',
    'hourly': [{'time': '0',
      'tempC': '-9',
      'tempF': '16',
      'windspeedMiles': '12',
      'windspeedKmph': '19',
      'winddirDegree': '298',
      'winddir16Point': 'WNW',
      'weatherCode': '113',
      'weatherIconUrl': [{'value': 'https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'}],
      'weatherDesc': [{'value': 'Clear'}],
      'precipMM': '0.0',
      'precipInches': '0.0',
      'humidity': '56',
      'vis

In [ ]:
weather_data = data_js['data']['weather']
weather_data 

[{'date': '2015-01-05',
  'astronomy': [{'sunrise': '08:06 AM',
    'sunset': '05:38 PM',
    'moonrise': '06:30 PM',
    'moonset': '08:03 AM',
    'moon_phase': 'Full Moon',
    'moon_illumination': '100'}],
  'maxtempC': '-8',
  'maxtempF': '18',
  'mintempC': '-12',
  'mintempF': '11',
  'avgtempC': '-9',
  'avgtempF': '15',
  'totalSnow_cm': '0.0',
  'sunHour': '8.7',
  'uvIndex': '2',
  'hourly': [{'time': '0',
    'tempC': '-9',
    'tempF': '16',
    'windspeedMiles': '12',
    'windspeedKmph': '19',
    'winddirDegree': '298',
    'winddir16Point': 'WNW',
    'weatherCode': '113',
    'weatherIconUrl': [{'value': 'https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'}],
    'weatherDesc': [{'value': 'Clear'}],
    'precipMM': '0.0',
    'precipInches': '0.0',
    'humidity': '56',
    'visibility': '10',
    'visibilityMiles': '6',
    'pressure': '1035',
    'pressureInches': '31',
    'cloudcover': '0',
    'HeatIndexC': '-9',
    'Hea

In [ ]:
moon = weather_data[0]['astronomy'][0]['moon_phase']
moon 

'Full Moon'